In [2]:
!ls ..

 blockchain	       data.zip		 Miniconda3-latest-Linux-x86_64.sh
 blockchain_code.zip   detectron.yml	 processed_pm25.csv
 blockchain_files      environment.yml	'quantile regression.ipynb'
 blockchain_GNN        match_data	 scene_reg
 csvfiles	       mend


In [14]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('../data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('../data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

['../data/names/Arabic.txt', '../data/names/Chinese.txt', '../data/names/Czech.txt', '../data/names/Dutch.txt', '../data/names/English.txt', '../data/names/French.txt', '../data/names/German.txt', '../data/names/Greek.txt', '../data/names/Irish.txt', '../data/names/Italian.txt', '../data/names/Japanese.txt', '../data/names/Korean.txt', '../data/names/Polish.txt', '../data/names/Portuguese.txt', '../data/names/Russian.txt', '../data/names/Scottish.txt', '../data/names/Spanish.txt', '../data/names/Vietnamese.txt']
Slusarski


In [16]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [17]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


In [18]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [19]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

In [20]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-2.9567, -2.8697, -2.8659, -2.8675, -2.9686, -2.8554, -2.8794, -2.9144,
         -2.8658, -2.8268, -2.8899, -2.8735, -2.9722, -2.9072, -2.9525, -2.8795,
         -2.9302, -2.7740]], grad_fn=<LogSoftmaxBackward0>)


In [21]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('Vietnamese', 17)


In [22]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Korean / line = Hwang
category = Vietnamese / line = Ha
category = Chinese / line = Yap
category = English / line = Lethbridge
category = Greek / line = Chrysanthopoulos
category = English / line = Nanton
category = Polish / line = Szwarc
category = Russian / line = Jirnov
category = Scottish / line = Macleod
category = Russian / line = Lezhankov


In [23]:
criterion = nn.NLLLoss()

In [24]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [25]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 5s) 2.6914 Mateus / Greek ✗ (Portuguese)
10000 10% (0m 10s) 0.9655 Nakata / Japanese ✓
15000 15% (0m 14s) 3.7075 Tejc / Chinese ✗ (Czech)
20000 20% (0m 20s) 1.2167 Iida / Japanese ✓
25000 25% (0m 25s) 3.0564 Genovese / French ✗ (Italian)
30000 30% (0m 30s) 0.9402 Sun / Korean ✓
35000 35% (0m 35s) 0.3416 Arishima / Japanese ✓
40000 40% (0m 39s) 0.5715 Dang / Vietnamese ✓
45000 45% (0m 45s) 1.5187 Ryskamp / Dutch ✓
50000 50% (0m 50s) 0.4671 Yun / Korean ✓
55000 55% (0m 55s) 2.0433 Koumans / Irish ✗ (Dutch)
60000 60% (1m 0s) 0.6469 Kieu / Vietnamese ✓
65000 65% (1m 5s) 2.9641 Stein / Dutch ✗ (German)
70000 70% (1m 10s) 1.3319 She / Korean ✗ (Chinese)
75000 75% (1m 15s) 1.2580 Glockner / German ✓
80000 80% (1m 20s) 1.0113 Armando / Spanish ✓
85000 85% (1m 25s) 1.2823 Beutel / German ✓
90000 90% (1m 30s) 0.4338 Chang / Korean ✓
95000 95% (1m 35s) 1.7077 Gomes / Dutch ✗ (Portuguese)
100000 100% (1m 40s) 0.2190 Kyritsis / Greek ✓


In [26]:
import torch.nn

In [28]:
rnn = nn.RNN(10, 20, 1)
input = torch.randn(5, 3, 10)
h0 = torch.randn(1, 3, 20)
output, hn = rnn(input, h0)


In [66]:
rnn = nn.RNN(10, 20, 1)
input = torch.randn(5, 3, 10)
# h0 = torch.randn(1, 3, 20)
output, hn = rnn(input)
hn

tensor([[[ 0.4567,  0.4685,  0.3684, -0.5473,  0.5217, -0.4581, -0.7762,
           0.0904,  0.2344,  0.1766, -0.6225, -0.5224,  0.5243,  0.9324,
           0.6850, -0.1740,  0.4469,  0.1863, -0.3867, -0.2271],
         [-0.6057, -0.3950, -0.0229, -0.2491, -0.0755,  0.6582,  0.3477,
          -0.0697, -0.1722, -0.5432, -0.3011, -0.0750,  0.0335, -0.4644,
          -0.4705,  0.5206, -0.3134, -0.1284,  0.3038,  0.3454],
         [-0.5861,  0.3617, -0.3680, -0.3918, -0.6417,  0.3259,  0.0376,
           0.7605,  0.2975, -0.9153, -0.5734, -0.2676, -0.0916, -0.9206,
           0.4656,  0.0398, -0.8018,  0.7255,  0.4777, -0.2697]]],
       grad_fn=<StackBackward0>)

In [72]:
resh = input[:,1,:].reshape(5,1,10)
output, hn = rnn(resh)
hn

tensor([[[-0.6057, -0.3950, -0.0229, -0.2491, -0.0755,  0.6582,  0.3477,
          -0.0697, -0.1722, -0.5432, -0.3011, -0.0750,  0.0335, -0.4644,
          -0.4705,  0.5206, -0.3134, -0.1284,  0.3038,  0.3454]]],
       grad_fn=<StackBackward0>)

In [61]:
output.shape

torch.Size([5, 3, 20])

In [62]:
hn

tensor([[[ 0.4758,  0.6339, -0.2396,  0.7967, -0.1029, -0.6692, -0.2747,
           0.1795,  0.3711, -0.5356, -0.0041,  0.0875,  0.5684, -0.2196,
          -0.6431,  0.2756,  0.1650,  0.2623,  0.2931, -0.1176],
         [-0.0205, -0.5409, -0.1812,  0.4569, -0.3888,  0.3819, -0.2954,
          -0.5021,  0.3025, -0.0082, -0.5537, -0.5464,  0.3223,  0.1146,
          -0.0335,  0.2482, -0.5348,  0.2603, -0.0939,  0.5771],
         [-0.5771, -0.8358,  0.0171, -0.2713,  0.0377,  0.5306,  0.3302,
          -0.1348,  0.1368, -0.0452, -0.5834, -0.0837, -0.1502, -0.2560,
          -0.2339, -0.4686, -0.2627, -0.3892, -0.5418,  0.3667]]],
       grad_fn=<StackBackward0>)

torch.Size([1, 3, 20])

In [37]:
from torch.nn.utils.rnn import pack_sequence
a = torch.tensor([1,2,3])
b = torch.tensor([4,5])
c = torch.tensor([6])
seq = pack_sequence([a, b, c])
seq

PackedSequence(data=tensor([1, 4, 6, 2, 5, 3]), batch_sizes=tensor([3, 2, 1]), sorted_indices=None, unsorted_indices=None)

In [43]:
seq[0][:seq[1][0]]

tensor([1, 4, 6])

In [48]:
import pytorch_forecasting.utils as utils

In [53]:
padded,lengths = utils.unpack_sequence(seq)

In [51]:
import torch.nn.utils.rnn as rnnutils

def unpack_sequence(packed_sequence, lengths):
    assert isinstance(packed_sequence, rnnutils.PackedSequence)
    head = 0
    trailing_dims = packed_sequence.data.shape[1:]
    unpacked_sequence = [torch.zeros(l, *trailing_dims) for l in lengths]
    # l_idx - goes from 0 - maxLen-1
    for l_idx, b_size in enumerate(packed_sequence.batch_sizes):
        for b_idx in range(b_size):
            unpacked_sequence[b_idx][l_idx] = packed_sequence.data[head]
            head += 1
    return unpacked_sequence

In [54]:
unpack_sequence(seq,lengths)

[tensor([1., 2., 3.]), tensor([4., 5.]), tensor([6.])]